In [1]:
import os
# LANGCHAIN_ENDPOINT = "http://127.0.0.1:5000"
LANGCHAIN_ENDPOINT = "http://localhost:8000" # Docker
# LANGCHAIN_ENDPOINT = "https://langchain-api-gateway-57eoxz8z.uc.gateway.dev" # GCP
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["LANGCHAIN_SESSION"] = "1"

from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI


In [2]:
import langchain

In [3]:
# Tracing demo with error

serp_api_key = os.getenv("SERPAPI_API_KEY")
os.environ["SERPAPI_API_KEY"] = "foo" # do this to force an error

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)

In [10]:
agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
) # Should error



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.
Action: Search
Action Input: "US Open men's tennis final 2019"

ValueError: Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key

In [11]:
# Tracing demo without error
os.environ["SERPAPI_API_KEY"] = serp_api_key

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
) # Should not error



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.
Action: Search
Action Input: "US Open men's tennis final 2019"
Observation: In a repeat of the 2019 Rogers Cup, Rafael Nadal won the men's singles title, defeating first time Grand Slam finalist Daniil Medvedev in a 5-set thriller for ...
Thought: I now need to find out Rafael Nadal's age
Action: Search
Action Input: "Rafael Nadal age"
Observation: 36 years
Thought: I now need to calculate 36 squared
Action: Calculator
Action Input: 36^2

> Entering new LLMMathChain chain...
36^2Answer: 1296
Observation: Answer: 1296
Thought: I now know the final answer
Final Answer: Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296.


"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [6]:
langchain.set_tracing_callback_manager(session=3)

In [8]:
agent.run(
    "Who won the US Open men's tennis final in 2022? What is his age raised to the second power?"
)



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's tennis final in 2022 and then calculate his age raised to the second power.
Action: Search
Action Input: "US Open men's tennis final 2022"

ValueError: Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key

In [9]:
llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
llm("Tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

In [10]:
import requests

r = requests.get(f"{LANGCHAIN_ENDPOINT}/sessions")
r.json()

[{'start_time': '2023-01-12T18:38:24.684746', 'extra': None, 'id': 1},
 {'start_time': '2023-01-12T18:39:19.314883', 'extra': None, 'id': 2}]

In [7]:
SharedLangChainTracer().load_session(1)

TracerSession(id=1, start_time=datetime.datetime(2023, 1, 6, 5, 13, 58, 726404, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')), extra={})

In [8]:
SharedLangChainTracer().load_session(2) # Expect exception -- this session does not exist

TracerException: Failed to load session 2

In [9]:
r = requests.get(f"{LANGCHAIN_ENDPOINT}/chain-runs?id=1")
r.json()

{'id': 1,
 'start_time': 1673010838.850066,
 'end_time': 1673010840.998612,
 'extra': {},
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'session_id': 1,
 'error': 'Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key',
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': None,
 'child_runs': [{'id': 2,
   'start_time': 1673010838.850123,
   'end_time': 1673010840.802959,
   'extra': {},
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'session_id': 1,
   'error': None,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "U

In [10]:
r = requests.get(f"{LANGCHAIN_ENDPOINT}/llm-runs?top_level=True&session_id=1") # Should be empty -- no top level llm runs
r.json()

[]

In [11]:
r = requests.get(f"{LANGCHAIN_ENDPOINT}/tool-runs?session_id=2") # Should be empty -- nothing for session id 2
r.json()

[]

In [12]:
r = requests.get(f"{LANGCHAIN_ENDPOINT}/chain-runs")
print(f"{len(r.json())} total chain runs")
r.json()

9 total chain runs


[{'id': 1,
  'start_time': '2023-01-06T05:13:58.850066',
  'end_time': '2023-01-06T05:14:00.998612',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'error': 'Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key',
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': None,
  'child_runs': [{'id': 2,
    'start_time': '2023-01-06T05:13:58.850123',
    'end_time': '2023-01-06T05:14:00.802959',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'error': None,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his

In [13]:
r = requests.get(f"{LANGCHAIN_ENDPOINT}/chain-runs?top_level=True")
print(f"{len(r.json())} total top level runs")
r.json()

2 total top level runs


[{'id': 1,
  'start_time': '2023-01-06T05:13:58.850066',
  'end_time': '2023-01-06T05:14:00.998612',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'error': 'Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key',
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': None,
  'child_runs': [{'id': 2,
    'start_time': '2023-01-06T05:13:58.850123',
    'end_time': '2023-01-06T05:14:00.802959',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'error': None,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his

In [9]:
import requests
r = requests.get(f"{LANGCHAIN_ENDPOINT}/llm-runs")
r.json()

[{'id': 1,
  'start_time': '2023-01-08T05:34:45.581979',
  'end_time': '2023-01-08T05:34:47.790858',
  'extra': {},
  'execution_order': 3,
  'serialized': {'name': 'OpenAI'},
  'session_id': 1,
  'error': None,
  'prompts': ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Who won the US Open men's tennis final in 2019? What is his age raised 